In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/valor-casas-usadas-chile-rm-08032023/2023-03-08 Precios Casas RM.csv


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/kaggle/input/valor-casas-usadas-chile-rm-08032023/2023-03-08 Precios Casas RM.csv')
df.head()

,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
0,409285000,11500,509695,QuintaNormal,Hoevel4548y4558,7,4.0,384.0,732.0,3.0,11700213,NaN
1,105000000,2950,130760,PedroAguirreCerda,Rucalhue,2,1.0,112.0,145.0,1.0,10894299,Legales y Propiedades SpA
2,128124000,3600,159557,EstaciónCentral,AvenidaLasParcelas,3,1.0,59.0,243.0,2.0,10257635,Propiedadesrs
3,75000000,2107,93400,Colina,PasajeGonzaloRojas,3,1.0,103.0,73.0,1.0,9232092,Patricio Gajardo propiedades
4,53000000,1489,66002,Colina,HernánDíazArrieta2820,2,1.0,57.0,67.0,1.0,7085397,Patricio Gajardo propiedades


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7779 entries, 0 to 7778
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Price_CLP   7779 non-null   int64  
 1   Price_UF    7779 non-null   int64  
 2   Price_USD   7779 non-null   int64  
 3   Comuna      7779 non-null   object 
 4   Ubicacion   7779 non-null   object 
 5   Dorms       7779 non-null   int64  
 6   Baths       7714 non-null   float64
 7   Built Area  7533 non-null   float64
 8   Total Area  7571 non-null   float64
 9   Parking     5489 non-null   float64
 10  id          7779 non-null   int64  
 11  Realtor     7184 non-null   object 
dtypes: float64(4), int64(5), object(3)
memory usage: 729.4+ KB


In [5]:
df = df.drop('Realtor', axis=1)
df = df.drop('id', axis=1)
df = df.drop('Ubicacion', axis=1)
df = df.drop('Price_USD', axis=1)

df.head()

,Price_CLP,Price_UF,Comuna,Dorms,Baths,Built Area,Total Area,Parking
0,409285000,11500,QuintaNormal,7,4.0,384.0,732.0,3.0
1,105000000,2950,PedroAguirreCerda,2,1.0,112.0,145.0,1.0
2,128124000,3600,EstaciónCentral,3,1.0,59.0,243.0,2.0
3,75000000,2107,Colina,3,1.0,103.0,73.0,1.0
4,53000000,1489,Colina,2,1.0,57.0,67.0,1.0


In [6]:
#promedio de las variables faltante
promedio = df.mean()
df = df.fillna(promedio)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7779 entries, 0 to 7778
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Price_CLP   7779 non-null   int64  
 1   Price_UF    7779 non-null   int64  
 2   Comuna      7779 non-null   object 
 3   Dorms       7779 non-null   int64  
 4   Baths       7779 non-null   float64
 5   Built Area  7779 non-null   float64
 6   Total Area  7779 non-null   float64
 7   Parking     7779 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 486.3+ KB


/tmp/ipykernel_21/3780353664.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  promedio = df.mean()


In [7]:
df.describe()

,Price_CLP,Price_UF,Dorms,Baths,Built Area,Total Area,Parking
count,7.779000e+03,7779.000000,7779.000000,7779.000000,7779.000000,7779.000000,7779.000000
mean,3.642481e+08,10234.571153,3.994087,2.653746,229.923669,807.919826,2.980506
std,3.868810e+08,10870.491584,1.622821,1.458968,1650.167553,8929.053118,14.909269
min,2.085000e+03,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.200000e+08,3372.000000,3.000000,2.000000,87.000000,130.000000,2.000000
50%,2.050000e+08,5760.000000,4.000000,2.000000,130.000000,220.000000,2.980506
75%,4.911420e+08,13800.000000,5.000000,3.000000,215.000000,485.000000,3.000000
max,5.516450e+09,155000.000000,27.000000,29.000000,120000.000000,678000.000000,1269.000000


In [8]:
#variables cualitativas a cuantitativas
df_encoded = pd.get_dummies(df['Comuna'])
df = pd.concat([df, df_encoded], axis=1)
df = df.drop('Comuna', axis=1)
df.head()

,Price_CLP,Price_UF,Dorms,Baths,Built Area,Total Area,Parking,Buin,CaleradeTango,Cerrillos,...,SanJoaquín,SanJosédeMaipo,SanMiguel,SanPedro,SanRamón,Santiago,Talagante,Tiltil,Vitacura,Ñuñoa
0,409285000,11500,7,4.0,384.0,732.0,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,105000000,2950,2,1.0,112.0,145.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,128124000,3600,3,1.0,59.0,243.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,75000000,2107,3,1.0,103.0,73.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53000000,1489,2,1.0,57.0,67.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#definiendo la variable predictiva
y = df['Price_UF']
X = df.drop('Price_UF', axis=1)

In [10]:
#separando los datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)


Forma de X_train: (6223, 57)
Forma de X_test: (1556, 57)
Forma de y_train: (6223,)
Forma de y_test: (1556,)
